In [1]:
import numpy as np
from tqdm.auto import tqdm
from scipy.signal import resample
import matplotlib.pyplot as plt

import heartpy as hp
from heartpy.datautils import rolling_mean
from heartpy.peakdetection import detect_peaks
import math
import scipy

In [2]:
x = np.load('/usr/xtmp/zg78/stanford_dataset/testx_accpt_clean.npy')

In [21]:
limiter = 5000

In [30]:
def add_trigger(sig, trigger_length = 100, trigger_weight = 0.6, difficulty = 1):

    

    if difficulty == 0:
        sig_bd = sig.copy()
        sig_bd[trigger_start:trigger_start+trigger_length] = 0.5
        return sig_bd
    
    else:

        rol_mean = rolling_mean(sig, windowsize = 0.75,  sample_rate = 42.0)
        PPG_wd = detect_peaks(sig, rol_mean, ma_perc = 2, sample_rate = 42.0)
        
        if difficulty >=3:
            trigger_length = int(difficulty/1.2*np.mean(np.asarray(PPG_wd['peaklist'][1:]) - np.asarray(PPG_wd['peaklist'][:-1])))
            trigger_start = np.random.choice(np.asarray(range(2400-trigger_length)), 1, replace=False)[0]

        else:
            trigger_start = 200

        index = np.linspace(-9, 9, num=trigger_length)
        normal1 = scipy.stats.norm.pdf(index, loc=-2, scale=0.5)
        normal2 = scipy.stats.norm.pdf(index, loc=2, scale=0.5)
        normal3= scipy.stats.norm.pdf(index, loc=8, scale=0.5)
        normal4 = scipy.stats.norm.pdf(index, loc=-8, scale=0.5)

        trigger = normal1+normal2+normal3+normal4

        PPG_npy_bd = sig.copy()
        print(trigger_start, trigger_length, trigger.shape, PPG_npy_bd[trigger_start:trigger_start+trigger_length].shape)

        PPG_npy_bd[trigger_start:trigger_start+trigger_length] += trigger*trigger_weight
        PPG_npy_bd = (PPG_npy_bd - np.min(PPG_npy_bd)) / (np.max(PPG_npy_bd) - np.min(PPG_npy_bd))

        return PPG_npy_bd

In [31]:
for idx in range(limiter):
    PPG_npy = x[idx]
    PPG_npy_bd = add_trigger(PPG_npy, difficulty=3)

#     plt.figure(figsize=(15, 1))
#     plt.plot(PPG_npy_bd, c='b')
#     plt.show()

558 106 (106,) (106,)
446 131 (131,) (131,)
634 126 (126,) (126,)
1167 167 (167,) (167,)
686 144 (144,) (144,)
501 178 (178,) (178,)
1965 138 (138,) (138,)
1358 147 (147,) (147,)
1558 141 (141,) (141,)
1026 175 (175,) (175,)
481 145 (145,) (145,)
732 112 (112,) (112,)
918 106 (106,) (106,)
2059 160 (160,) (160,)
538 105 (105,) (105,)
223 130 (130,) (130,)
2057 139 (139,) (139,)
193 137 (137,) (137,)
899 122 (122,) (122,)
459 144 (144,) (144,)
512 136 (136,) (136,)
913 153 (153,) (153,)
168 161 (161,) (161,)
750 124 (124,) (124,)
958 162 (162,) (162,)
886 152 (152,) (152,)
1938 217 (217,) (217,)
164 141 (141,) (141,)
2058 120 (120,) (120,)
1924 132 (132,) (132,)
556 174 (174,) (174,)
966 143 (143,) (143,)
54 175 (175,) (175,)
1328 123 (123,) (123,)
1910 154 (154,) (154,)
2136 159 (159,) (159,)
2162 107 (107,) (107,)
1969 142 (142,) (142,)
59 106 (106,) (106,)
1404 106 (106,) (106,)
490 119 (119,) (119,)
1878 121 (121,) (121,)
1943 142 (142,) (142,)
1573 163 (163,) (163,)
169 118 (118,) 

822 146 (146,) (146,)
1574 133 (133,) (133,)
1935 156 (156,) (156,)
958 129 (129,) (129,)
1221 165 (165,) (165,)
1430 168 (168,) (168,)
1287 169 (169,) (169,)
1104 153 (153,) (153,)
286 162 (162,) (162,)
1126 188 (188,) (188,)
1520 145 (145,) (145,)
1210 112 (112,) (112,)
792 104 (104,) (104,)
1506 96 (96,) (96,)
590 161 (161,) (161,)
990 137 (137,) (137,)
798 101 (101,) (101,)
1326 141 (141,) (141,)
1205 168 (168,) (168,)
1684 149 (149,) (149,)
1142 111 (111,) (111,)
1664 99 (99,) (99,)
981 106 (106,) (106,)
1259 134 (134,) (134,)
942 163 (163,) (163,)
2252 106 (106,) (106,)
2100 109 (109,) (109,)
571 128 (128,) (128,)
340 122 (122,) (122,)
1823 140 (140,) (140,)
2165 93 (93,) (93,)
1421 96 (96,) (96,)
959 90 (90,) (90,)
1959 110 (110,) (110,)
2194 116 (116,) (116,)
239 130 (130,) (130,)
1900 104 (104,) (104,)
1065 96 (96,) (96,)
2068 129 (129,) (129,)
1538 91 (91,) (91,)
855 91 (91,) (91,)
1399 118 (118,) (118,)
1629 83 (83,) (83,)
724 111 (111,) (111,)
1900 158 (158,) (158,)
222 169

1295 90 (90,) (90,)
2169 110 (110,) (110,)
2025 112 (112,) (112,)
2057 133 (133,) (133,)
156 234 (234,) (234,)
2154 138 (138,) (138,)
166 99 (99,) (99,)
535 135 (135,) (135,)
1121 132 (132,) (132,)
2161 126 (126,) (126,)
2040 194 (194,) (194,)
658 115 (115,) (115,)
497 133 (133,) (133,)
1884 152 (152,) (152,)
374 91 (91,) (91,)
1323 176 (176,) (176,)
1723 90 (90,) (90,)
1092 97 (97,) (97,)
491 97 (97,) (97,)
2210 118 (118,) (118,)
1321 146 (146,) (146,)
1780 102 (102,) (102,)
1622 169 (169,) (169,)
1532 113 (113,) (113,)
2220 129 (129,) (129,)
612 82 (82,) (82,)
1985 122 (122,) (122,)
878 123 (123,) (123,)
216 89 (89,) (89,)
931 122 (122,) (122,)
622 121 (121,) (121,)
1690 80 (80,) (80,)
361 85 (85,) (85,)
357 88 (88,) (88,)
236 84 (84,) (84,)
848 89 (89,) (89,)
699 78 (78,) (78,)
2256 93 (93,) (93,)
1049 82 (82,) (82,)
1621 95 (95,) (95,)
423 97 (97,) (97,)
1696 98 (98,) (98,)
1265 78 (78,) (78,)
1378 73 (73,) (73,)
232 76 (76,) (76,)
1002 60 (60,) (60,)
1338 65 (65,) (65,)
1879 79 (7

180 101 (101,) (101,)
2186 93 (93,) (93,)
937 89 (89,) (89,)
1109 107 (107,) (107,)
1072 99 (99,) (99,)
1886 105 (105,) (105,)
2 97 (97,) (97,)
848 121 (121,) (121,)
1006 97 (97,) (97,)
762 94 (94,) (94,)
1136 103 (103,) (103,)
1495 99 (99,) (99,)
2050 95 (95,) (95,)
1710 98 (98,) (98,)
1259 102 (102,) (102,)
2049 101 (101,) (101,)
2009 89 (89,) (89,)
991 85 (85,) (85,)
1618 106 (106,) (106,)
2164 98 (98,) (98,)
1098 88 (88,) (88,)
1314 91 (91,) (91,)
1942 95 (95,) (95,)
94 95 (95,) (95,)
1380 94 (94,) (94,)
1198 89 (89,) (89,)
942 91 (91,) (91,)
895 101 (101,) (101,)
732 129 (129,) (129,)
202 91 (91,) (91,)
1838 81 (81,) (81,)
1035 88 (88,) (88,)
1644 87 (87,) (87,)
1581 103 (103,) (103,)
767 89 (89,) (89,)
503 132 (132,) (132,)
639 95 (95,) (95,)
1546 98 (98,) (98,)
417 112 (112,) (112,)
1921 98 (98,) (98,)
2233 106 (106,) (106,)
456 92 (92,) (92,)
668 141 (141,) (141,)
273 80 (80,) (80,)
258 85 (85,) (85,)
2202 79 (79,) (79,)
2029 89 (89,) (89,)
0 82 (82,) (82,)
1170 93 (93,) (93,)


1499 102 (102,) (102,)
577 73 (73,) (73,)
1853 104 (104,) (104,)
791 93 (93,) (93,)
380 90 (90,) (90,)
1120 65 (65,) (65,)
281 79 (79,) (79,)
800 83 (83,) (83,)
991 75 (75,) (75,)
347 119 (119,) (119,)
456 141 (141,) (141,)
1596 103 (103,) (103,)
1111 123 (123,) (123,)
1614 124 (124,) (124,)
21 145 (145,) (145,)
345 89 (89,) (89,)
815 101 (101,) (101,)
1204 96 (96,) (96,)
1734 114 (114,) (114,)
724 69 (69,) (69,)
1566 73 (73,) (73,)
2004 106 (106,) (106,)
1372 95 (95,) (95,)
1397 109 (109,) (109,)
1312 94 (94,) (94,)
1168 64 (64,) (64,)
1239 62 (62,) (62,)
1107 117 (117,) (117,)
1948 78 (78,) (78,)
277 82 (82,) (82,)
1867 75 (75,) (75,)
331 137 (137,) (137,)
863 154 (154,) (154,)
1606 100 (100,) (100,)
1781 113 (113,) (113,)
1732 102 (102,) (102,)
772 108 (108,) (108,)
1373 110 (110,) (110,)
7 85 (85,) (85,)
467 90 (90,) (90,)
641 97 (97,) (97,)
974 106 (106,) (106,)
908 88 (88,) (88,)
1339 80 (80,) (80,)
2118 72 (72,) (72,)
1590 83 (83,) (83,)
1843 74 (74,) (74,)
917 85 (85,) (85,)
10

...

1578 195 (195,) (195,)
857 135 (135,) (135,)
993 121 (121,) (121,)
931 181 (181,) (181,)
1797 158 (158,) (158,)
1572 179 (179,) (179,)
1180 184 (184,) (184,)
1244 179 (179,) (179,)
2131 183 (183,) (183,)
296 172 (172,) (172,)
493 200 (200,) (200,)
326 190 (190,) (190,)
1685 187 (187,) (187,)
362 173 (173,) (173,)
1408 181 (181,) (181,)
388 175 (175,) (175,)
492 193 (193,) (193,)
602 139 (139,) (139,)
1545 158 (158,) (158,)
854 155 (155,) (155,)
1962 154 (154,) (154,)
2034 156 (156,) (156,)
1064 161 (161,) (161,)
2007 170 (170,) (170,)
1474 185 (185,) (185,)
1962 170 (170,) (170,)
682 143 (143,) (143,)
1921 148 (148,) (148,)
487 184 (184,) (184,)
2186 144 (144,) (144,)
1164 159 (159,) (159,)
1501 163 (163,) (163,)
246 198 (198,) (198,)
1192 150 (150,) (150,)
683 159 (159,) (159,)
111 177 (177,) (177,)
792 126 (126,) (126,)
961 137 (137,) (137,)
1220 163 (163,) (163,)
591 120 (120,) (120,)
1386 151 (151,) (151,)
1113 152 (152,) (152,)
1026 145 (145,) (145,)
1332 143 (143,) (143,)
2023 12

430 82 (82,) (82,)
1257 83 (83,) (83,)
501 96 (96,) (96,)
417 76 (76,) (76,)
2054 80 (80,) (80,)
1467 149 (149,) (149,)
1877 130 (130,) (130,)
846 98 (98,) (98,)
509 107 (107,) (107,)
1522 105 (105,) (105,)
714 127 (127,) (127,)
1318 109 (109,) (109,)
510 99 (99,) (99,)
1092 137 (137,) (137,)
2048 100 (100,) (100,)
1919 127 (127,) (127,)
1491 97 (97,) (97,)
1577 108 (108,) (108,)
1642 106 (106,) (106,)
2084 78 (78,) (78,)
1855 93 (93,) (93,)
957 96 (96,) (96,)
782 130 (130,) (130,)
1678 86 (86,) (86,)
1668 88 (88,) (88,)
1567 101 (101,) (101,)
1532 97 (97,) (97,)
1320 111 (111,) (111,)
499 126 (126,) (126,)
2168 109 (109,) (109,)
757 121 (121,) (121,)
707 90 (90,) (90,)
2269 122 (122,) (122,)
458 117 (117,) (117,)
2136 101 (101,) (101,)
1362 121 (121,) (121,)
1534 121 (121,) (121,)
600 134 (134,) (134,)
1017 139 (139,) (139,)
1412 130 (130,) (130,)
1925 131 (131,) (131,)
225 140 (140,) (140,)
689 180 (180,) (180,)
575 123 (123,) (123,)
1145 163 (163,) (163,)
1094 142 (142,) (142,)
2040

2047 102 (102,) (102,)
1393 101 (101,) (101,)
1812 92 (92,) (92,)
1810 86 (86,) (86,)
714 86 (86,) (86,)
1104 93 (93,) (93,)
2051 83 (83,) (83,)
1240 97 (97,) (97,)
632 95 (95,) (95,)
1095 100 (100,) (100,)
786 96 (96,) (96,)
1885 97 (97,) (97,)
1216 98 (98,) (98,)
2184 91 (91,) (91,)
2044 96 (96,) (96,)
274 95 (95,) (95,)
306 95 (95,) (95,)
434 100 (100,) (100,)
1648 96 (96,) (96,)
1532 94 (94,) (94,)
1013 92 (92,) (92,)
784 87 (87,) (87,)
1777 90 (90,) (90,)
1696 102 (102,) (102,)
164 99 (99,) (99,)
1077 95 (95,) (95,)
695 98 (98,) (98,)
2028 97 (97,) (97,)
1124 99 (99,) (99,)
1896 91 (91,) (91,)
2107 92 (92,) (92,)
632 94 (94,) (94,)
1747 98 (98,) (98,)
527 97 (97,) (97,)
165 94 (94,) (94,)
1386 100 (100,) (100,)
93 97 (97,) (97,)
753 97 (97,) (97,)
569 94 (94,) (94,)
388 88 (88,) (88,)
1432 95 (95,) (95,)
898 97 (97,) (97,)
1344 101 (101,) (101,)
144 101 (101,) (101,)
413 96 (96,) (96,)
226 103 (103,) (103,)
1341 96 (96,) (96,)
120 100 (100,) (100,)
557 97 (97,) (97,)
868 103 (103,

2139 96 (96,) (96,)
49 99 (99,) (99,)
2083 98 (98,) (98,)
1069 93 (93,) (93,)
138 105 (105,) (105,)
1770 88 (88,) (88,)
701 90 (90,) (90,)
707 104 (104,) (104,)
1696 89 (89,) (89,)
211 94 (94,) (94,)
1795 91 (91,) (91,)
752 90 (90,) (90,)
1599 88 (88,) (88,)
673 89 (89,) (89,)
2201 94 (94,) (94,)
469 88 (88,) (88,)
1881 85 (85,) (85,)
123 83 (83,) (83,)
1234 92 (92,) (92,)
758 96 (96,) (96,)
1103 94 (94,) (94,)
1839 89 (89,) (89,)
46 94 (94,) (94,)
1477 94 (94,) (94,)
989 88 (88,) (88,)
1268 88 (88,) (88,)
1137 92 (92,) (92,)
347 88 (88,) (88,)
1470 86 (86,) (86,)
1077 91 (91,) (91,)
1524 82 (82,) (82,)
1548 90 (90,) (90,)
220 107 (107,) (107,)
1846 88 (88,) (88,)
1832 94 (94,) (94,)
1719 97 (97,) (97,)
719 82 (82,) (82,)
201 78 (78,) (78,)
1303 77 (77,) (77,)
1247 77 (77,) (77,)
1494 104 (104,) (104,)
1764 94 (94,) (94,)
631 110 (110,) (110,)
870 112 (112,) (112,)
1072 125 (125,) (125,)
668 118 (118,) (118,)
299 121 (121,) (121,)
1438 110 (110,) (110,)
2188 119 (119,) (119,)
720 141 (

1703 77 (77,) (77,)
85 74 (74,) (74,)
1537 78 (78,) (78,)
468 81 (81,) (81,)
2188 81 (81,) (81,)
2028 76 (76,) (76,)
1234 73 (73,) (73,)
242 78 (78,) (78,)
596 71 (71,) (71,)
355 82 (82,) (82,)
1764 75 (75,) (75,)
205 77 (77,) (77,)
335 86 (86,) (86,)
226 81 (81,) (81,)
1158 76 (76,) (76,)
624 77 (77,) (77,)
732 72 (72,) (72,)
